In [1]:
import pandas as pd
from pathlib import Path
import bs4 as bs
import requests
import re
import unidecode
import us

DATA_DIR = Path('.')/'..'/'..'/'data'
pd.set_option('display.max_rows', 500)

In [2]:
##### Map name:abbr for states
# Initial mapping
STATES_DICT = us.states.mapping('name', 'abbr')
# Remove all unwanted states
territories = ['American Samoa', 'Dakota', 'District of Columbia', 'Guam','Northern Mariana Islands', 
               'Orleans', 'Philippine Islands', 'Puerto Rico', 'Virgin Islands']
for t in territories:
    del STATES_DICT[t]

    
##### Map parties of interest; all others will be cast to NaN #####
PARTY_DICT = {'republican' : 'r', 'democratic' : 'd'}


##### Regex shortcuts #####
NAME = r"[\w\s\-\.\'\,]"

def clean_name(series):
    """
    Remove following punctuation commonly found in names: - . , '
    Also strip names of accents
    """
    result = series.copy()
    
    punc_list = ['-', '.', '\'', ',']
    for punc in punc_list:
        result = result.str.replace(punc, '')
    series = series.apply(unidecode.unidecode)
    
    return result

# Senate Names

### Names 2010, 2012, 2014, 2016, 2018

In [3]:
def scrape_senate_names(year):
    """Works for 2010, 2012, 2014, 2016, 2018."""
    RACE_TYPE = 'senate'
    source = requests.get(f'https://en.wikipedia.org/wiki/{year}_United_States_Senate_elections')
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    
    names = []
    parties = []
    race_types = []
    years = []
    states = []
    
    # iterate through all tables with "Candidates"
    for table in soup.find_all('table', class_="wikitable sortable"):
        if 'Candidates' not in [h.text.strip() for h in table.find_all('th', text=True)]:
            continue
        
        for tr in table.find_all('tr')[2:]:
            state = re.match(r"([\w\s]+).*", tr.find('td').text).groups()[0].strip()
                
            # find candidate names
            candidates = tr.find_all('td')[-1]
            candidates_text = re.findall(rf"\w*?{NAME}+?\(\w+?\)", candidates.text)
            for candidate in candidates_text:
                matcher = re.match(rf"\w*?({NAME}+)\s\((\w+)\)", candidate.lower())
                if matcher:
                    name, party = matcher.groups()
                    names.append(name)
                    parties.append(party)
                    race_types.append(RACE_TYPE)
                    years.append(year)
                    states.append(state)
    
    # return df
    df = pd.DataFrame({'name': names, 
                       'party': parties,
                       'race_type': race_types,
                       'year': years,
                       'state': states})
    df['name'] = clean_name(df['name'])
    df['state'] = df['state'].map(STATES_DICT)
    df = df.dropna(subset=['state'])
    df['party'] = df['party'].map(PARTY_DICT)
    df.loc[df['party'].isnull(), 'party'] = 'o'
    
    return df

In [4]:
senate_names_2010 = scrape_senate_names(2010)
senate_names_2012 = scrape_senate_names(2012)
senate_names_2014 = scrape_senate_names(2014)
senate_names_2016 = scrape_senate_names(2016)
senate_names_2018 = scrape_senate_names(2018)

### Concatenate

In [5]:
senate_names = pd.concat([senate_names_2010, senate_names_2012, senate_names_2014, senate_names_2016, senate_names_2018],
                        ignore_index=True)

In [6]:
print('Shape:', senate_names.shape)
print('\nNull values:')
print(senate_names.isnull().sum())
senate_names

Shape: (589, 5)

Null values:
name         0
party        0
race_type    0
year         0
state        0
dtype: int64


,name,party,race_type,year,state
0,scott brown,r,senate,2010,MA
1,martha coakley,d,senate,2010,MA
2,joseph l kennedy,o,senate,2010,MA
3,chris coons,d,senate,2010,DE
4,christine odonnell,r,senate,2010,DE
5,glenn miller,o,senate,2010,DE
6,james rash,o,senate,2010,DE
7,mark kirk,r,senate,2010,IL
8,alexi giannoulias,d,senate,2010,IL
9,lealan jones,o,senate,2010,IL


In [7]:
# senate_names.to_csv(DATA_DIR/'cleaned'/'candidate_parties_senate.csv', index=False)

# Senate Election Results